# WAVE tutorial: a smart home

WAVE is platform for delegable authorization. We are going to test it out by looking at some use cases in a smart home context. You can also use WAVE for securing other things, like APIs, but this is a fun tutorial to cover the basics.

## Getting started

The lines below will "boot" your house. There is a home server that acts as the root of authority for your house and all its devices. Give your house a unique nickname, as later you will be accessing data from other people's houses. 

In [1]:
from tut import *
my_unique_nickname="john-smith"
# wave is a connection to the WAVE daemon, that you will use for authorization actions
# homeserver is the control interface to your home server, allowing you to request permissions etc.
# mqtt is a connection to the rise camp MQTT server that routes all commands and sensor information 
wave, homeserver, mqtt = Initialize(my_unique_nickname)

## Creating your entity

An *entity* represents a person or device. Your home server has an entity which is the root of permissions in the house, but every occupant of the house also needs their own entity so they can be granted permissions to interact with the house. Go ahead and create your entity by running the command below. 

Note that usually the command would be `entity = wave.CreateEntity(wv.CreateEntityParams())` but to make sure you don't lose your entity if you restart the notebook, we have made a wrapper that saves it to disk

In [2]:
entity, _ = createOrLoadEntity(wave, "myEntity")

# a perspective is the form of an entity you use in calls to WAVE
perspective = wv.Perspective(entitySecret=wv.EntitySecret(DER=entity.SecretDER))

## Proofs of permission

Let's try control the house with this brand new entity. To do so, we need to form a *proof of permissions*:


In [3]:
proof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    # our secret entity perspective
    perspective=perspective,
    # the domain of authority we are accessing
    namespace=homeserver.namespace(),
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

# lets check if that worked:
if proof.error.code != 0:
    print ("Proof building failed:", proof.error.message)
else:
    print ("Proof building succeeeded")

Proof building succeeeded


If this is the first time you've run the notebook, the above will fail. This is because this newly created entity has not been granted any permissions! Why should it be able to control the house? In real life, you would now communicate with a person/service that has the permissions you want and give them your entity hash. With this, they can grant you permissions. We will emulate that with a simple function call:

In [4]:
homeserver.grant_permissions_to(entity.hash)

That grants you permissions to everything within the house, later we will do a permission grant from scratch, so you can see how it is done. For now, lets try building a proof again:

In [5]:
proof = wave.BuildRTreeProof(wv.BuildRTreeProofParams(
    # our secret entity perspective
    perspective=perspective,
    # the domain of authority we are accessing
    namespace=homeserver.namespace(),
    # what we want to prove
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        )
    ]
))

if proof.error.code != 0:
    print ("Proof building failed:", proof.error.message)
else:
    print ("Proof building succeeeded")

Proof building succeeeded


This should succeed. We can now attach this proof to a command we send to the light, and it will obey the command because it can verify we are authorized. For this demo, the smart home devices rendered at the top of the notebook are listening for commands on an MQTT topic unique to your nickname.

In [8]:
homeserver.render()

In [9]:
_ = mqtt.publish(my_unique_nickname+"/smarthome/light/control",
#                      you can also use {"state":"off"}
                  composeMessage(proof, {"state":"on"}))

john-smith/smarthome/light/control
got light command



If you scroll to the top of the notebook, you should see that your light has turned on. Let's walk through how you would decrypt and validate a message that you receive. Let's subscribe to the thermostat report topic:

In [17]:
def thermostat_cb(msg):
    # first decrypt the message
    resp = wave.DecryptMessage(wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= msg.payload,
        resyncFirst= True))
    if resp.error.code != 0:
        print ("dropping thermostat message:", resp.error.message)
        return
    # then break it up into proof + body
    proof, body = decomposeMessage(resp.content)
    
    # now validate the proof
    resp = wave.VerifyProof(wv.VerifyProofParams(
        proofDER=proof,
        requiredRTreePolicy=wv.RTreePolicy(
            namespace=homeserver.namespace(),
            statements=[wv.RTreePolicyStatement(
                permissionSet=smarthome_pset,
                permissions=["write"],
                resource="smarthome/light/report",
            )]
        )
    ))
    if resp.error.code != 0:
        print ("dropping message: ", resp.error.message)
        
    # the proof is valid!
    print ("got:", body)
    # now split it into proof and payload:
# todo change back to thermostat
mqtt.subscribe(my_unique_nickname+"/smarthome/light/report", thermostat_cb)

subscribing to john-smith/smarthome/light/report


In [18]:


# test encryption/decryption
msg = "hello world"
uri = "smarthome/light/result"
encryptResult = wave.EncryptMessage(
    wv.EncryptMessageParams(
        namespace=homeserver.namespace(), 
        resource=uri, 
        content=bytes(msg,"utf8")))
print(encryptResult)

# try decrypt it
decryptResult1 = wave.DecryptMessage(
    wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= encryptResult.ciphertext,
        resyncFirst= True))

print (decryptResult1)

# grant permissions from the home server to us
homeserver.grant_permissions_to(entity.hash)

# try decrypt again
decryptResult2 = wave.DecryptMessage(
    wv.DecryptMessageParams(
        perspective= perspective,
        ciphertext= encryptResult.ciphertext,
        resyncFirst= True))

print (decryptResult2)

ciphertext: "(\202\004\021\006\n+\006\001\004\001\203\217U\002\005\240\202\004\0010\202\003\375\004\033\330Y=\t\177\"\367\207x2\014\235tD\346Z\312ap\305\246l\274e\222>G0\202\003\332(\202\003\326\006\n+\006\001\004\001\203\217U\020\002\240\202\003\3060\202\003\302\004\202\002\3410!\320\007\360\361\303\336\315C\027\346\204>\370E\245p\343Fj\177C\220f2\245A\327\253\241\316\235Gh.\347\265\006,\375.5\357\tA\345\032\200\216lyl\255\223(@\021%\023\334\256\333\246}l&\211\177\266\247t\273-\214/p\213_6\3370\367t\247\353\263\014\372\247\306\021(\216\357R\246\202J\234|\236K\002\3423\226W\241\273\200\334\271j\201\252o\017\330SZ\361B\374r\013kbG\306o\313\335\213\010\033\305\034U9w -UH\255\264\330\356\033\346=U\002g !\2323(]\223\226\374\025\344H\325\251\333\373qX\241*/MbQ\256\233\310\212\360e\357\330\013\350\242\024\320\020\033\332\327\324\363p\204|\t\363\235\340\204\247W\361|\226hW\2654\004\004\3732\307q\035\266\267CIL~\372HRX\217\020\325sX\212\336\227f\202\341.\346\020\221\365$g2\212W\351\370\2660\22

In [17]:

entity = agent.CreateEntity(wv.CreateEntityParams())
if entity.error.code != 0:
    raise Exception(entity.error)
agent.PublishEntity(wv.PublishEntityParams(DER=entity.PublicDER))
perspective=wv.Perspective(
    entitySecret=wv.EntitySecret(DER=entity.SecretDER)
)
proof = agent.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=hs.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["actuate"],
            resource="smarthome/light/control",
        )
    ]
))
# should have no permissions
print ("first attempt without permissions:")
print (proof.error)

hs.grant_permissions_to(entity.hash)
proof2 = agent.BuildRTreeProof(wv.BuildRTreeProofParams(
    perspective=perspective,
    namespace=hs.namespace(),
    resyncFirst=True,
    statements=[
        wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["write"],
            resource="smarthome/light/control",
        ),
      wv.RTreePolicyStatement(
            permissionSet=smarthome_pset,
            permissions=["read"],
            resource="smarthome/light/report",
        )
    ]
))
if proof2.error.code != 0:
    raise Exception(proof2.error)

NameError: name 'agent' is not defined

In [ ]:
packed = pack_payload(proof2.proofDER, json.dumps({'state': 'on'}))
client.publish("michael/smarthome/light/control", packed)

In [ ]:
packed = pack_payload(proof3.proofDER, json.dumps({'hsp': 68}))
client.publish("michael/smarthome/thermostat/control", packed)

In [ ]:
homeserver.render()